In [1]:
# ============================================================
# 🔥 LGBM 모델링 (pair_train_v2 + FE v2 기반)
#   - train: pair_train_v2 (self-supervised)
#   - test : sample_submission 기준으로 feature 생성 후 예측
# ============================================================

import pandas as pd
import numpy as np
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor


In [2]:

# ------------------------------------------------------------
# 0️⃣ Path 설정 & 데이터 로드
# ------------------------------------------------------------
BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR.parents[1] / "data"

PAIR_PATH = DATA_DIR / "processed" / "pair_train_v2.csv"
FE_PATH   = DATA_DIR / "processed" / "v2_features_basic.csv"
MONTHLY_PATH = DATA_DIR / "processed" / "train_monthly.csv"
SUB_PATH  = DATA_DIR / "raw" / "sample_submission.csv"

print("📂 Loading files...")
pair_train = pd.read_csv(PAIR_PATH)
FE = pd.read_csv(FE_PATH)
monthly = pd.read_csv(MONTHLY_PATH)
sub = pd.read_csv(SUB_PATH)

print(" pair_train:", pair_train.shape)
print(" FE        :", FE.shape)
print(" monthly   :", monthly.shape)
print(" sample_sub:", sub.shape)



📂 Loading files...
 pair_train: (343354, 15)
 FE        : (100, 8)
 monthly   : (3734, 8)
 sample_sub: (9900, 3)


In [3]:
# ------------------------------------------------------------
# 1️⃣ train feature / target 구성
#   - pair_train_v2에는 이미:
#     [b_t, b_t_1, a_t_lag, max_corr, best_lag] + FE 수치 컬럼 + target
#   - 문자열 ID(leading_item_id, following_item_id, lead_hs3)는 일단 제외
# ------------------------------------------------------------

# 어떤 컬럼들이 있는지 한번 확인
print("\n🔍 pair_train_v2 columns:")
print(pair_train.columns.tolist())

# FE 쪽에도 어떤 컬럼 있는지 확인
print("\n🔍 FE columns:")
print(FE.columns.tolist())

# 모델에 넣을 수치형 feature만 선택
#  - ID류(leading_item_id, following_item_id, lead_hs3)는 제외
#  - object, string 컬럼은 우선 제외 (원하면 나중에 Label Encoding으로 추가 가능)
exclude_cols = ["leading_item_id", "following_item_id", "lead_hs3"]
feature_cols = []

for c in pair_train.columns:
    if c in exclude_cols:
        continue
    if c == "target":
        continue
    # 수치형만 사용
    if np.issubdtype(pair_train[c].dtype, np.number):
        feature_cols.append(c)

print("\n🧱 사용 feature 컬럼:")
print(feature_cols)

X = pair_train[feature_cols].copy()
y = pair_train["target"].values

# 혹시 NaN 있으면 0으로 채움 (특히 corr_hs3, multi_leader_lag 등)
X = X.fillna(0.0)

print("\n✅ X shape:", X.shape)
print("✅ y shape:", y.shape)




🔍 pair_train_v2 columns:
['leading_item_id', 'following_item_id', 'b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 'target', 'zero_ratio', 'volatility', 'cv', 'lead_hs3', 'best_lag_hs3', 'corr_hs3', 'multi_leader_lag']

🔍 FE columns:
['item_id', 'zero_ratio', 'volatility', 'cv', 'lead_hs3', 'best_lag_hs3', 'corr_hs3', 'multi_leader_lag']

🧱 사용 feature 컬럼:
['b_t', 'b_t_1', 'a_t_lag', 'max_corr', 'best_lag', 'zero_ratio', 'volatility', 'cv', 'best_lag_hs3', 'corr_hs3', 'multi_leader_lag']

✅ X shape: (343354, 11)
✅ y shape: (343354,)


In [4]:
# ------------------------------------------------------------
# 2️⃣ Train / Valid 분할
# ------------------------------------------------------------
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\n✂️ Split:")
print(" X_train:", X_train.shape, " X_valid:", X_valid.shape)

# ------------------------------------------------------------
# 3️⃣ LGBMRegressor 설정 & 학습
#   (verbose 관련 에러 방지: fit에는 verbose 인자 안 넘김)
# ------------------------------------------------------------

model = LGBMRegressor(
    boosting_type="gbdt",
    num_leaves=127,
    max_depth=12,
    learning_rate=0.03,
    n_estimators=2000,

    subsample=0.8,
    colsample_bytree=0.8,
    subsample_freq=1,

    min_child_samples=40,
    min_child_weight=1e-3,

    reg_alpha=0.0,
    reg_lambda=10.0,  # 중요!!

    random_state=42,
    n_jobs=-1,
    verbose=-1,
)


print("\n🔥 Training LGBMRegressor...")
model.fit(
    X_train,
    y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="l2",
)

# ------------------------------------------------------------
# 4️⃣ 검증 성능 (RMSE)
#   - sklearn 구버전 대응: squared=False 대신 제곱근 직접
# ------------------------------------------------------------
y_pred_valid = model.predict(X_valid)
mse = mean_squared_error(y_valid, y_pred_valid)
rmse = mse ** 0.5

print(f"\n📏 Validation RMSE: {rmse:.4f}")




✂️ Split:
 X_train: (274683, 11)  X_valid: (68671, 11)

🔥 Training LGBMRegressor...

📏 Validation RMSE: 1569442.1592


In [5]:
# ------------------------------------------------------------
# 5️⃣ 제출용 Feature 생성
#   - sample_submission.csv 기준으로 (leading, following) 쌍에 대해
#     b_t, b_t_1, a_t_lag, max_corr, best_lag + FE(v2) 구성
# ------------------------------------------------------------

print("\n===== Build test features for submission =====")

# 5-1. monthly에서 ym 컬럼 만들고 pivot_item 생성
monthly["hs3"] = monthly["hs4"].astype(str).str[:3]
monthly["ym"] = pd.to_datetime(
    monthly["year"].astype(str) + "-" + monthly["month"].astype(str).str.zfill(2)
)

pivot_item = monthly.pivot(
    index="item_id", columns="ym", values="total_value"
).fillna(0.0)

print("pivot_item shape:", pivot_item.shape)

months = sorted(pivot_item.columns)
# 마지막 3개월 (확인용)
print("last 3 months:", months[-3:])

t_last = len(months) - 1   # 마지막 달 index (ex: 2025-07)
t_prev = len(months) - 2   # 그 전 달 index

# 5-2. pair_train_v2에서 각 (leading, following) 쌍의 best_lag, max_corr 가져오기
pair_meta = (
    pair_train[["leading_item_id", "following_item_id", "best_lag", "max_corr"]]
    .drop_duplicates()
)

print("pair_meta shape:", pair_meta.shape)

# 5-3. sample_submission과 pair_meta merge
test_df = sub.merge(
    pair_meta,
    on=["leading_item_id", "following_item_id"],
    how="left",
)

print("sample_submission shape:", sub.shape)
print("\nMerged test_df shape:", test_df.shape)

# 5-4. following_item_id 기준으로 FE(v2) merge
test_df = test_df.merge(
    FE,
    left_on="following_item_id",
    right_on="item_id",
    how="left",
)

test_df = test_df.drop(columns=["item_id"], errors="ignore")

print("After FE merge, test_df shape:", test_df.shape)

# 5-5. (b_t, b_t_1, a_t_lag) 계산
bt_list = []
bt1_list = []
alag_list = []

for row in test_df.itertuples():
    leader = row.leading_item_id
    follower = row.following_item_id
    lag = int(row.best_lag) if not np.isnan(row.best_lag) else 1

    # leader/follower가 pivot_item에 없으면 0으로
    if (leader not in pivot_item.index) or (follower not in pivot_item.index):
        bt_list.append(0.0)
        bt1_list.append(0.0)
        alag_list.append(0.0)
        continue

    xs = pivot_item.loc[leader].values.astype(float)
    ys = pivot_item.loc[follower].values.astype(float)

    b_t = ys[t_last]
    b_t_1 = ys[t_prev]

    # t_last - lag가 음수가 되지 않도록 방어
    if t_last - lag >= 0:
        a_t_lag = xs[t_last - lag]
    else:
        a_t_lag = 0.0

    bt_list.append(b_t)
    bt1_list.append(b_t_1)
    alag_list.append(a_t_lag)

test_df["b_t"] = bt_list
test_df["b_t_1"] = bt1_list
test_df["a_t_lag"] = alag_list

print("\n✅ Added (b_t, b_t_1, a_t_lag)")
print(test_df[["leading_item_id", "following_item_id", "b_t", "b_t_1", "a_t_lag"]].head())

# 5-6. 학습 때 사용한 feature_cols와 동일하게 X_test 구성
#      (혹시 pair_train에만 있던 feature가 있다면, 없는 컬럼은 0으로 채움)
X_test = pd.DataFrame()

for c in feature_cols:
    if c in test_df.columns:
        X_test[c] = test_df[c]
    else:
        # test 쪽에 없는 feature는 0으로 채워서라도 컬럼 맞추기
        X_test[c] = 0.0

X_test = X_test.fillna(0.0)

print("\nX_test shape:", X_test.shape)



===== Build test features for submission =====
pivot_item shape: (100, 43)
last 3 months: [Timestamp('2025-05-01 00:00:00'), Timestamp('2025-06-01 00:00:00'), Timestamp('2025-07-01 00:00:00')]
pair_meta shape: (8930, 4)
sample_submission shape: (9900, 3)

Merged test_df shape: (9900, 5)
After FE merge, test_df shape: (9900, 12)

✅ Added (b_t, b_t_1, a_t_lag)
  leading_item_id following_item_id       b_t      b_t_1   a_t_lag
0        DEWLVASR          ELQGMQWE  455972.0  1045639.0  358930.0
1        DEWLVASR          AHMDUILJ  101317.0    89479.0  446042.0
2        DEWLVASR          XIPPENFQ  183079.0    68313.0  229898.0
3        DEWLVASR          FTSVTTSR  246916.0   129389.0  408887.0
4        DEWLVASR          XMKRPGLB  474247.0  1771172.0  361589.0

X_test shape: (9900, 11)


In [6]:

# ------------------------------------------------------------
# 6️⃣ 예측 & 제출 파일 생성
# ------------------------------------------------------------
y_test_pred = model.predict(X_test)

# 음수 방지 + 정수화
y_test_pred = np.maximum(0, y_test_pred)
y_test_pred = np.rint(y_test_pred).astype(int)

submission = sub.copy()
submission["value"] = y_test_pred

print("\n제출 파일 미리보기:")
print(submission.head())

OUT_SUB_PATH = BASE_DIR / "lgbm_pair_v2_submission.csv"
submission.to_csv(OUT_SUB_PATH, index=False)

print(f"\n💾 Submission saved to: {OUT_SUB_PATH}")



제출 파일 미리보기:
  leading_item_id following_item_id    value
0        DEWLVASR          ELQGMQWE   551804
1        DEWLVASR          AHMDUILJ   100692
2        DEWLVASR          XIPPENFQ   134657
3        DEWLVASR          FTSVTTSR   211018
4        DEWLVASR          XMKRPGLB  1003350

💾 Submission saved to: /data/ephemeral/home/notebooks/modeling/lgbm_pair_v2_submission.csv
